## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-04-12-45-00 +0000,nypost,NJ governor’s race spending to shatter records...,https://nypost.com/2025/10/04/us-news/record-m...
1,2025-10-04-12-40-32 +0000,nyt,Israel and Hamas Say They’ll Work With Trump’s...,https://www.nytimes.com/2025/10/04/world/middl...
2,2025-10-04-12-38-59 +0000,bbc,Hopes and fears in Gaza and Israel over potent...,https://www.bbc.com/news/videos/c8exnwe00y3o?a...
3,2025-10-04-12-37-30 +0000,cbc,Russian strike hits passenger train in Ukraine...,https://www.cbc.ca/news/world/russia-drone-ukr...
4,2025-10-04-12-35-34 +0000,nypost,"Millions of demonstrators strike, march across...",https://nypost.com/2025/10/04/world-news/prote...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,54
15,gaza,18
12,hamas,17
485,court,15
332,shutdown,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
106,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...,143
199,2025-10-03-20-03-20 +0000,nypost,Hamas agrees to release all Israeli hostages p...,https://nypost.com/2025/10/03/world-news/hamas...,138
31,2025-10-04-10-44-52 +0000,wapo,High stakes for Gaza peace talks as Hamas acce...,https://www.washingtonpost.com/world/2025/10/0...,128
84,2025-10-04-03-10-45 +0000,nypost,"Hamas internal rift over hostages, weapons cou...",https://nypost.com/2025/10/03/world-news/hamas...,127
146,2025-10-03-22-22-09 +0000,nyt,Trump Calls on Israel to Stop Bombing Gaza Aft...,https://www.nytimes.com/2025/10/03/us/politics...,124


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
106,143,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...
21,64,2025-10-04-12-00-00 +0000,wsj,Many government leaders have had the chance to...,https://www.wsj.com/economy/trade/south-korea-...
276,57,2025-10-03-14-42-11 +0000,latimes,Supreme Court will decide if gun owners may ca...,https://www.latimes.com/politics/story/2025-10...
197,50,2025-10-03-20-27-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...
220,48,2025-10-03-18-23-42 +0000,nypost,Terrorist Jihad Al-Shamie was on bail for rape...,https://nypost.com/2025/10/03/world-news/uk-te...
45,47,2025-10-04-09-49-00 +0000,wsj,Japan Set for First Female Prime Minister Amid...,https://www.wsj.com/world/asia/japan-prime-min...
207,40,2025-10-03-19-37-00 +0000,wsj,Trump ordered another lethal military strike o...,https://www.wsj.com/politics/national-security...
82,36,2025-10-04-03-28-52 +0000,nypost,50 Cent trolls longtime rival Sean ‘Diddy’ Com...,https://nypost.com/2025/10/03/entertainment/50...
126,34,2025-10-03-23-23-33 +0000,nypost,Andrew Cuomo raises $400K in just 2 days follo...,https://nypost.com/2025/10/03/us-news/andrew-c...
114,33,2025-10-04-00-13-38 +0000,nypost,NJ teen revealed dark obsession with one of hi...,https://nypost.com/2025/10/03/us-news/nj-teen-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
